In [ ]:
import pandas as pd
from openai import OpenAI

# put your API key and (optionally) your organization here
client = OpenAI(
    api_key='',
    organization='',
)

# load neurips data
df = pd.read_pickle('data/neurips.pkl')

# add category column
df.insert(5, 'category', 'None')

In [ ]:
# annotate all papers
for i in range(len(df)):
    print(i)
    paper = df.iloc[i]

    prompt = [
        {
            'role': 'user',
            'content': f"The following paper was published in NeurIPS:\nTitle: {paper['title']}\nAuthors: {paper['authors']}\nAbstract: {paper['abstract']}\n\nCategorize this paper into one of the following categories: ['Algorithms', 'Applications', 'Data, Challenges, Implementations, and Software', 'Deep Learning', 'Neuroscience and Cognitive Science', 'Optimization', 'Probabilistic Methods', 'Reinforcement Learning and Planning', 'Theory', 'Safety and Alignment']. Output only the category name in the format <category name>. If the paper does not fit into any of these categories, output 'Other'."
        }
    ]

    result = client.chat.completions.create(
        messages=prompt,
        model='gpt-3.5-turbo',
    )

    msg = result.choices[0].message.content

    df.loc[i, 'category'] = msg[msg.find('<')+1:msg.find('>')]

In [ ]:
# clean up
categories = ['Algorithms', 'Applications', 'Data, Challenges, Implementations, and Software', 'Deep Learning', 'Neuroscience and Cognitive Science', 'Optimization', 'Probabilistic Methods', 'Reinforcement Learning and Planning', 'Theory', 'Safety and Alignment', 'Other']

for i in range(len(df)):
    if df.iloc[i]['category'] not in categories:
        for category in categories:
            if category in df.iloc[i]['category']:
                print(f"fixed: {df.iloc[i]['category']} -> {category}")
                df.at[i, 'category'] = category
                break
            elif df.iloc[i]['category'] in category:
                print(f"fixed: {df.iloc[i]['category']} -> {category}")
                df.at[i, 'category'] = category
                break
        else:
            if df.iloc[i]['category'] in 'Security and Privacy':
                print(f"fixed: {df.iloc[i]['category']} -> Safety and Alignment")
                df.at[i, 'category'] = 'Safety and Alignment'
            else:
                print(f"not fixed: {df.iloc[i]['category']}")
                df.at[i, 'category'] = 'Other'


In [ ]:
# save data
df.to_pickle('data/nips.pkl')